`CTE`: `Common Table Expression`。优点：
- 提高复杂查询的可读性
- 能够创建递归查询，即引用自身的查询
- 与窗口函数结合使用

```
WITH cte_name (column1, column2, ...) AS (
    -- CTE query
    SELECT ...
)
-- Main query using the CTE
SELECT ...
FROM cte_name;
```

In [2]:
import common.ipynb_importer
from db.pg.pg_00_common import *

cursor = pg_connect()

importing Jupyter notebook from E:\sourcecode\keep_learning\db\pg\pg_00_common.ipynb


In [3]:
sql = """
WITH action_films AS (
  SELECT 
    f.title, 
    f.length 
  FROM 
    film f 
    INNER JOIN film_category fc USING (film_id) 
    INNER JOIN category c USING(category_id) 
  WHERE 
    c.name = 'Action'
) 
SELECT * FROM action_films;
"""

run_sql(cursor, sql)

                     title  length
0             Amadeus Holy     113
1          American Circus     129
2       Antitrust Tomatoes     168
3            Ark Ridgemont      68
4      Barefoot Manchurian     129
..                     ...     ...
59         Uprising Uptown     174
60  Waterfront Deliverance      61
61           Werewolf Lola      79
62            Women Dorado     126
63            Worst Banger     185

[64 rows x 2 columns]


In [4]:
sql = """
WITH cte_rental AS (
  SELECT 
    staff_id, 
    COUNT(rental_id) rental_count 
  FROM 
    rental 
  GROUP BY 
    staff_id
) 
SELECT 
  s.staff_id, 
  first_name, 
  last_name, 
  rental_count 
FROM 
  staff s 
  INNER JOIN cte_rental USING (staff_id);
"""

run_sql(cursor, sql)

   staff_id first_name last_name  rental_count
0         1       Mike   Hillyer          8040
1         2        Jon  Stephens          8004


In [5]:
sql = """
WITH film_stats AS (
    -- CTE 1: Calculate film statistics
    SELECT
        AVG(rental_rate) AS avg_rental_rate,
        MAX(length) AS max_length,
        MIN(length) AS min_length
    FROM film
),
customer_stats AS (
    -- CTE 2: Calculate customer statistics
    SELECT
        COUNT(DISTINCT customer_id) AS total_customers,
        SUM(amount) AS total_payments
    FROM payment
)
-- Main query using the CTEs
SELECT
    ROUND((SELECT avg_rental_rate FROM film_stats), 2) AS avg_film_rental_rate,
    (SELECT max_length FROM film_stats) AS max_film_length,
    (SELECT min_length FROM film_stats) AS min_film_length,
    (SELECT total_customers FROM customer_stats) AS total_customers,
    (SELECT total_payments FROM customer_stats) AS total_payments;
"""

run_sql(cursor, sql)

  avg_film_rental_rate  max_film_length  min_film_length  total_customers  \
0                 2.98              185               46              599   

  total_payments  
0       61312.04  
